# Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

# How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

# Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import IPython
import cv2
import itertools
import numpy as np
import pandas as pd
import json
import base64
import datetime
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl
from matplotlib import pyplot as plt

In [ ]:
imp.reload(video);

# Define Pravega stream parameters

In [ ]:
gateway = 'hop-claudio-ubuntu18-desktop.solarch.lab.emc.com:54672'
scope = 'examples'
stream = 'object-detector-input-video'
#stream = 'object-detector-output-video'

# Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

# Build timestamp index
We want the from and to stream cuts for every frame.

In [ ]:
indexed_stream = IndexedStream(pravega_client, scope, stream)

In [ ]:
%%time
indexed_stream.build_index()

# Video Player

In [ ]:
class PravegaVideoPlayer():
    def __init__(self, indexed_stream):
        self.indexed_stream = indexed_stream
        
    def show(self, frame_number):
        video_frame = self.indexed_stream.get_single_video_frame_by_index(frame_number)
        timestamp = video_frame['timestamp']
        self.timestamp_label.value = '%s' % timestamp
        self.streamcut_widget.value = video_frame['from_stream_cut']
        plt.figure(figsize=(12,10))
        plt.imshow(opencv_image_to_mpl(video_frame['image_array']));

    def move_to_prev_frame(self):
        self.frame_number_widget.value = self.frame_number_widget.value - 1
        
    def move_to_next_frame(self):
        self.frame_number_widget.value = self.frame_number_widget.value + 1
    
    def interact(self):
        w = interactive(
            self.show, 
            frame_number=widgets.IntSlider(
                description='Frame',
                min=0, 
                max=len(self.indexed_stream.index_df)-1, 
                step=1, 
                value=0,
                style={'description_width': 'initial'}))
        self.widget = w
        self.frame_number_widget = w.children[0]
        self.output_widget = widgets.Output()
        self.timestamp_label = widgets.Text(description='Timestamp', disabled=True)
        self.streamcut_widget = widgets.Text(description='Stream Cut', disabled=True)
        
        prev_button = widgets.Button(description="<")
        prev_button.on_click(lambda b: self.move_to_prev_frame())
        next_button = widgets.Button(description=">")
        next_button.on_click(lambda b: self.move_to_next_frame())
        buttons = (prev_button, next_button)
        labels = (self.timestamp_label, self.streamcut_widget)
        image_widget = w.children[-1]
        
        w.children = (self.frame_number_widget,) + labels + buttons + (image_widget, self.output_widget)
        
        layout = w.layout
        w.layout.display = 'flex'
        w.layout.flex_flow = 'row wrap'
        w.layout.justify_content = 'flex-start'
        w.layout.align_items = 'flex-start'
        self.frame_number_widget.layout.width = '100%'        
        for child in labels:
            child.layout.width = '100%'
        for child in buttons:
            child.layout.width = '10%'
        image_widget.layout.width = '100%'
        self.output_widget.layout.width = '100%'
        
        display(self.widget)

In [ ]:
player = PravegaVideoPlayer(indexed_stream)

In [ ]:
player.interact()